# Câu hỏi: Liệu có thể phân loại được các bài báo theo từng chủ đề hay không?

---

## Import

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns # seaborn là thư viện được xây trên matplotlib, giúp việc visualization đỡ khổ hơn
import requests
import numpy as np
import pandas as pd
import time # để sleep chương trình
from bs4 import BeautifulSoup
from pyvi import ViTokenizer # thư viện NLP tiếng Việt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
#Mô hình phân lớp
from sklearn.naive_bayes import MultinomialNB

---

## Thu thập dữ liệu

Hàm `split_word` ở bên dưới có input:
- `articles`: là một dictionary với key là chủ đề bài báo, value: là một list nội dung các bài báo trong chủ đề đó.

Output: Trả về biến `articles` sau khi đã tách các từ trong văn bản

In [2]:
def split_word(df):
    for i in range(len(df['Nội dung văn bản'])):
        df['Nội dung văn bản'][i] =  ViTokenizer.tokenize(df['Nội dung văn bản'][i])
    return df

Đầu tiên ta sẽ lấy dữ liệu đã thu thập được từ file `data.csv`

In [3]:
data_df = pd.read_csv('DataLink.txt', index_col = 0)
data_df

,Nội dung văn bản,Chủ đề
0,Bắn thuốc mê di dời đàn khỉ trong khu dân cư...,thoi-su
1,Lớp học bằng ván gỗ cũ nát ở vùng cao Lạng S...,thoi-su
2,,thoi-su
3,Sài Gòn xuống 19 độ C Ảnh hưởng không khí lạ...,thoi-su
4,,thoi-su
...,...,...
2446,Lễ hội áo dài sắp diễn ra bên bờ sông Hương ...,du-lich
2447,Trên máy bay không phải chỗ nào cũng cho trẻ...,du-lich
2448,Cắm trại đón Giáng sinh trong sân bay Singap...,du-lich
2449,Saigontourist Group khuyến mại lớn dịp Giáng...,du-lich


Sau đó ta thực hiện tách từ trong dữ liệu bằng hàm `split_word`

In [4]:
data_df = split_word(data_df)
data_df

,Nội dung văn bản,Chủ đề
0,Bắn thuốc_mê di_dời đàn khỉ trong khu dân_cư S...,thoi-su
1,Lớp_học bằng ván gỗ cũ nát ở vùng_cao Lạng Sơn...,thoi-su
2,,thoi-su
3,Sài_Gòn xuống 19 độ C Ảnh_hưởng không_khí lạnh...,thoi-su
4,,thoi-su
...,...,...
2446,Lễ_hội áo_dài sắp diễn ra bên bờ sông Hương Th...,du-lich
2447,Trên máy_bay không phải chỗ nào cũng cho trẻ_e...,du-lich
2448,Cắm trại đón Giáng_sinh trong sân_bay Singapor...,du-lich
2449,Saigontourist_Group khuyến_mại lớn dịp Giáng_s...,du-lich


---

## Khám phá dữ liệu

In [5]:
data_df.head()

,Nội dung văn bản,Chủ đề
0,Bắn thuốc_mê di_dời đàn khỉ trong khu dân_cư S...,thoi-su
1,Lớp_học bằng ván gỗ cũ nát ở vùng_cao Lạng Sơn...,thoi-su
2,,thoi-su
3,Sài_Gòn xuống 19 độ C Ảnh_hưởng không_khí lạnh...,thoi-su
4,,thoi-su


In [6]:
#Dữ liệu có bao nhiêu dòng và bao nhiêu cột?
data_df.shape

(2451, 2)

In [7]:
# Dữ liệu có các dòng bị lặp không?
data_df.index.duplicated().sum()

0

In [8]:
#Kiểu dữ liệu của các cột
data_df.dtypes

Nội dung văn bản    object
Chủ đề              object
dtype: object

In [9]:
#Có dòng nào không lấy được nội dung văn bản không?
data_df['Nội dung văn bản'].isna().sum()

0

In [10]:
# Cột output có giá trị thiếu không?
data_df['Chủ đề'].isna().sum()

0

In [11]:
# Tỉ lệ các lớp trong cột output?
data_df['Chủ đề'].value_counts(normalize=True) * 100

suc-khoe      12.892697
giai-tri      12.811098
the-thao      12.647899
kinh-doanh    12.607099
du-lich       12.443900
the-gioi      12.443900
thoi-su       12.076703
giao-duc      12.076703
Name: Chủ đề, dtype: float64

Nhận xét: Tập dữ liệu phân bố khá đều.

Trước khi ta đi vào tiền xử lý dữ liệu, thì ta phải phân tích cấu trúc văn bản tiếng việt để giúp việc xử lý dữ liệu chính xác và hiệu quả hơn. Đầu tiên ta sẽ lấy lại nội dung của file data.csv lưu vào biến data_raw_df tức là dữ liệu trước khi ta tách từ, lúc này vẫn còn là những từ đơn, để tiện cho việc xem xét.

In [12]:
data_raw_df = pd.read_csv('DataLink.txt', index_col = 0)

Trong class `TfidfVectorizer` có 2 siêu tham số `min_df`và `max_df`. Ta sẽ đi tìm hiểu ý nghĩa của 2 siêu tham số trên xem thử chúng có ý nghĩa gì, và giúp được gì cho mô hình của chúng ta không!
- max_df được sử dụng để xóa các cụm từ xuất hiện quá thường xuyên, còn được gọi là "từ dừng cụ thể". Ví dụ:

    + max_df = 0.50 có nghĩa là "bỏ qua các thuật ngữ xuất hiện trong hơn 50% tài liệu".
    + max_df = 25 có nghĩa là "bỏ qua các thuật ngữ xuất hiện trong hơn 25 tài liệu".
    + max_df mặc định là 1.0, có nghĩa là "bỏ qua các thuật ngữ xuất hiện trong hơn 100% tài liệu". Do đó, cài đặt mặc định không bỏ qua bất kỳ thuật ngữ nào.

- min_df được sử dụng để xóa các cụm từ xuất hiện quá không thường xuyên. Ví dụ:

    + min_df = 0.01 có nghĩa là "bỏ qua các thuật ngữ xuất hiện trong ít hơn 1% tài liệu".
    + min_df = 5 có nghĩa là "bỏ qua các thuật ngữ xuất hiện trong ít hơn 5 tài liệu".
    + min_df mặc định là 1, có nghĩa là "bỏ qua các thuật ngữ xuất hiện trong ít hơn 1 tài liệu". Do đó, cài đặt mặc định không bỏ qua bất kỳ thuật ngữ nào.

Đầu tiên ta sẽ để các tham số trên là mặc định, tức là không xóa từ nào và in các từ này cùng với thông số IDF của chúng ra theo thứ tự idf giảm dần. Ta có thể hiểu nôm na là IDF càng cao thì tỉ lệ xuất hiện của nó trong toàn bộ các mẫu trong dữ liệu càng thấp, và ngược lại IDF càng thấp thì nó xuất hiện càng nhiều trong các mẫu trong dữ liệu, tức là gần như mẫu nào cũng có.

In [13]:
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 1))
tfidf.fit_transform(data_raw_df['Nội dung văn bản'])
feature_names = tfidf.get_feature_names()

Sau đó ta sẽ in ra cột IDF theo thứ tự giảm dần để xem thông số IDF của các từ trong dữ liệu

In [14]:
document_vector=tfidf.idf_

#print the scores 
df = pd.DataFrame(document_vector, index=feature_names, columns=["IDF"])
df.sort_values(by=["IDF"],ascending=False)

,IDF
00,8.111512
millan,8.111512
member,8.111512
membership,8.111512
membershop,8.111512
...,...
cho,1.211789
có,1.193807
của,1.182485
trong,1.164055


Nhận xét: Ta có thể thấy những từ ở trên cùng (IDF cao) là những từ không quá phổ biến trong tiếng việt (Thậm chí là tiếng anh hoặc con số). Còn những từ ở dưới cùng của bảng (IDF thấp) thì lại là những từ thông dụng mà gần như ở đâu cũng thấy (Có thể coi là stopword).

Theo ta thấy thì với những từ quá hiếm xuất hiện sẽ có thể dẫn đến tình trạng overfit cho mô hình, ngược lại các stopword lại làm cho mô hình kém hiệu quả. Vì vậy ta muốn bỏ đi những từ có IDF quá cao (từ đặc biệt) và có IDF quá thấp (stopword) để tăng tính chính xác của mô hình. Và nhờ 2 tham số `max_df` và `min_df` của `TfidfVectorizer` sẽ giúp ta loại bỏ những từ đó đi. Vậy việc ta cần làm bây giờ là cần xác định tham số `max_df` và `min_df` nào là tốt nhất!

Trước khi tiền xử lý ta thử tương tự như vừa rồi với `max_df=0.6` và `min_df=5` xem thử ta đã xóa được những từ nào

In [15]:
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 1), min_df=5, max_df=0.6)
tfidf.fit_transform(data_raw_df['Nội dung văn bản'])
feature_names = tfidf.get_feature_names()

document_vector=tfidf.idf_

#print the scores 
df = pd.DataFrame(document_vector, index=feature_names, columns=["IDF"])
df.sort_values(by=["IDF"],ascending=False)

,IDF
ửng,7.012900
bikini,7.012900
billboard,7.012900
grace,7.012900
viet,7.012900
...,...
nhất,1.555444
như,1.553314
trước,1.547657
thành,1.542031


Như ta thấy là ta đã xóa được rất nhiều từ và đúng theo ý muốn của ta. Bây giờ ta sẽ bắt tay vào các bước tiền xử lý và xây dựng mô hình!

---

## Tiền xử lý dữ liệu

Xóa đi những dòng có số kí tự ít hơn 100 (Có thể có những dòng không lấy được dữ liệu vì không giống định dạng mẫu)

In [16]:
data_df.drop(data_df[data_df['Nội dung văn bản'].map(len) < 1000].index, inplace = True)
#Dữ liệu sau khi xóa.
data_df.shape

(2069, 2)

Ta sẽ chuyển cột Output `Chủ đề` thành dạng số

In [17]:
encoder = LabelEncoder()
data_df['Chủ đề'] = encoder.fit_transform(data_df['Chủ đề'])
data_df

,Nội dung văn bản,Chủ đề
0,Bắn thuốc_mê di_dời đàn khỉ trong khu dân_cư S...,7
1,Lớp_học bằng ván gỗ cũ nát ở vùng_cao Lạng Sơn...,7
3,Sài_Gòn xuống 19 độ C Ảnh_hưởng không_khí lạnh...,7
5,Ông lão 72 tuổi vào đại_học TP HCMÔng Nguyễn V...,7
6,Hơn 1 000 gia_súc chết rét 1 080 con trâu_bò d...,7
...,...,...
2443,UNESCO công_nhận văn_hóa bán hàng rong là di_s...,0
2445,Vẻ đẹp nguyên_sơ của Hồ Tràm Bà_Rịa Vũng TàuCu...,0
2446,Lễ_hội áo_dài sắp diễn ra bên bờ sông Hương Th...,0
2449,Saigontourist_Group khuyến_mại lớn dịp Giáng_s...,0


---

## Tách dữ liệu + Mô hình hóa

In [18]:
X_df = data_df['Nội dung văn bản']
y_df = data_df['Chủ đề']
#Tách dữ liệu thành tập train và tập test
X_train, X_val, y_train, y_val = train_test_split(X_df, y_df, test_size=0.3,stratify = y_df, random_state=0)

Tạo Pipeline

In [19]:
process_pipeline = Pipeline([('tfidf',TfidfVectorizer(analyzer='word', ngram_range=(2, 3))),
                             ('classifier',MultinomialNB())])

### Tìm mô hình tốt nhất

Tìm giá trị tốt nhất của hai siêu tham số `alpha`, `max_df` và `min_df`
- Tham số `alpha` trong MultinomialNB.
Khi tính xác suất có trường hợp p = 0. Do đó tử số sẽ cộng cho α để tránh trường hợp đó.
- Tham số `max_df` trong TfidfVectorizer.
Được sử dụng để xóa các cụm từ xuất hiện quá thường xuyên.
- Tham số `min_df` trong TfidfVectorizer.
Được sử dụng để xóa các cụm từ xuất hiện quá không thường xuyên.

In [20]:
train_errs = []
val_errs = []
alphas = [0.05, 0.1, 0.5, 1]
min_df_s = [2, 3, 5, 7] # min_df=2 tức là bỏ qua các từ xuất hiện trong ít hơn 2 mẫu dữ liệu
max_df_s = [0.5, 0.6, 0.7, 0.8] # max_df=0.5 tức là bỏ qua các từ xuất hiện trong nhiều hơn 50% dữ liệu
best_val_err = float('inf'); 
best_alpha = None;
best_min_df = None
best_max_df = None

for alpha in alphas:
    for min_df in min_df_s:
        for max_df in max_df_s:
            process_pipeline.set_params(tfidf__min_df = min_df,
                                        tfidf__max_df = max_df,
                                        classifier__alpha = alpha)
            process_pipeline.fit(X_train, y_train)
            train_score = (1 - process_pipeline.score(X_train, y_train))*100
            val_score = (1 - process_pipeline.score(X_val, y_val))*100
            train_errs.append(train_score)
            val_errs.append(val_score)
            if float(val_score) < best_val_err:
                best_val_err = val_score
                best_alpha = alpha
                best_max_df = max_df
                best_min_df = min_df

Lấy `best_alpha`, `best_max_df`và `best_min_df` để huấn luyện mô hình

In [21]:
process_pipeline.set_params(tfidf__max_df = best_max_df,
                            tfidf__min_df = best_min_df,
                            classifier__alpha = best_alpha)
process_pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.5, min_df=3, ngram_range=(2, 3))),
                ('classifier', MultinomialNB(alpha=0.05))])

Kết quả dự đoán tập huấn luyện

In [22]:
train_predictions = process_pipeline.predict(X_train)
accuracy_score(train_predictions, y_train)

0.9958563535911602

Kết quả dự đoán tập validation

In [23]:
val_predictions = process_pipeline.predict(X_val)
accuracy_score(val_predictions, y_val)

0.92914653784219

### Mô hình tốt nhất

In [24]:
process_pipeline.fit(X_df, y_df)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.5, min_df=3, ngram_range=(2, 3))),
                ('classifier', MultinomialNB(alpha=0.05))])

### Sử dụng mô hình tốt nhất để dự đoán tập test

Dữ liệu test được lấy từ trang web https://vietnamnet.vn/

Đầu tiên ta lấy dữ liệu tập test đã thu thập được từ file `test.csv`

In [25]:
data_test_df = pd.read_csv('Test.txt', index_col = 0)
data_test_df

,Nội dung văn bản,Chủ đề
0,Công an huyện Thanh Trì Hà Nội hôm nay 15 1 ...,thoi-su
1,Lắp đặt hệ thống điện mặt trời áp mái để phục ...,thoi-su
2,“Trong mọi trường hợp người Việt Nam nhập cản...,thoi-su
3,Công an thị xã Sa Pa Lào Cai đang làm rõ nam...,thoi-su
4,Bị can Nguyễn Thành Mỹ đồng phạm với ông Lê T...,thoi-su
...,...,...
680,Làng Tiebele trở thành địa điểm du lịch hấp dẫ...,du-lich
681,Mới đây công trình này bị liệt trong danh sác...,du-lich
682,Ở tuổi 14 bạn bè và gia đình Nikhil Kamath có...,du-lich
683,“Tôi thực sự thích làm những việc bình thường ...,du-lich


Sau đó ta thực hiện tách từ trong dữ liệu bằng hàm `split_word` tương tự như ở trên

In [26]:
data_test_df = split_word(data_test_df)
data_test_df

,Nội dung văn bản,Chủ đề
0,Công_an huyện Thanh_Trì Hà_Nội hôm_nay 15 1 ch...,thoi-su
1,Lắp_đặt hệ_thống điện mặt_trời áp mái để phục_...,thoi-su
2,“ Trong mọi trường_hợp người Việt_Nam nhập_cản...,thoi-su
3,Công_an thị_xã Sa_Pa Lào_Cai đang làm rõ nam t...,thoi-su
4,Bị_can Nguyễn Thành Mỹ đồng_phạm với ông Lê_Tấ...,thoi-su
...,...,...
680,Làng Tiebele trở_thành địa_điểm du_lịch hấp_dẫ...,du-lich
681,Mới_đây công_trình này bị liệt trong danh_sách...,du-lich
682,Ở tuổi 14 bạn_bè và gia_đình Nikhil Kamath có_...,du-lich
683,“ Tôi thực_sự thích làm những việc bình_thường...,du-lich


Xóa đi những dòng không lấy được dữ liệu

In [27]:
data_test_df.drop(data_test_df[data_test_df['Nội dung văn bản'].map(len) == 0].index, inplace = True)
data_test_df.shape

(685, 2)

Cuối cùng ta dự đoán kết quả tập dữ liệu test và tính tỉ lệ chính xác

In [28]:
test_X_df = data_test_df['Nội dung văn bản']
test_y_df = data_test_df['Chủ đề']
test_y_df = encoder.transform(test_y_df)
test_predictions = process_pipeline.predict(test_X_df)
accuracy_score(test_predictions, test_y_df)

0.8145985401459854